docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-extract -p /opt/car.lua /data/centro-oeste-latest.osm.pbf || echo "osrm-extract failed" 


docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-partition /data/centro-oeste-latest.osrm || echo "osrm-partition failed"
docker run -t -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-customize /data/centro-oeste-latest.osrm || echo "osrm-customize failed"

docker run -t -i -p 5000:5000 -v "${PWD}:/data" ghcr.io/project-osrm/osrm-backend osrm-routed --algorithm mld /data/centro-oeste-latest.osrm

curl "http://127.0.0.1:5000/route/v1/driving/-15.801795,-47.896361;13.385983,52.496891?steps=true"

In [3]:
import requests
import folium
import polyline
import numpy as np
import pandas as pd

In [4]:
url = "http://127.0.0.1:5000/route/v1/driving/-48.043619,-16.000091;-48.043619,-15.842141"
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'routes': [{'geometry': 'f|s`BpufdHnEtC}E|ItX|RmJ`Pei@m@_uDioEqLzRecB~sA_v@xb@unJnwBgjAgb@ePr@w_@se@_Pwf@{qAdj@oWuj@vGgCvBvE',
   'legs': [{'steps': [],
     'summary': '',
     'weight': 1698.7,
     'duration': 1698.7,
     'distance': 23945.3}],
   'weight_name': 'routability',
   'weight': 1698.7,
   'duration': 1698.7,
   'distance': 23945.3}],
 'waypoints': [{'hint': 'BGoQgApqEIAXAAAAAAAAAHQAAAB9AAAAW4IbQQAAAACHxkFC8otRQhcAAAAAAAAAdAAAAH0AAADCAAAA6OUi_bfdC_-d6SL9pdsL_wEALwJgC8Vo',
   'distance': 117.30249163,
   'name': '',
   'location': [-48.044568, -15.999561]},
  {'hint': 'v6UTgP___389AAAAQQAAAAAAAABZAAAAaPGjQiO-qUAAAAAAN6e4Qj0AAABBAAAAAAAAAFkAAADCAAAAm-ki_ahEDv-d6SL9o0QO_wAAXw9gC8Vo',
   'distance': 0.593319936,
   'name': 'Avenida Arniqueira',
   'location': [-48.043621, -15.842136]}]}

In [5]:
polyline.decode('f|s`BpufdHnEtC}E|ItX|RmJ`Pei@m@_uDioEqLzRecB~sA_v@xb@unJnwBgjAgb@ePr@w_@se@_Pwf@{qAdj@oWuj@vGgCvBvE')

[(-15.99956, -48.04457),
 (-16.0006, -48.04532),
 (-15.99949, -48.04707),
 (-16.0036, -48.05026),
 (-16.00177, -48.05299),
 (-15.99502, -48.05276),
 (-15.9659, -48.01943),
 (-15.96373, -48.02261),
 (-15.9477, -48.03621),
 (-15.9389, -48.04194),
 (-15.88007, -48.06122),
 (-15.86803, -48.05558),
 (-15.86528, -48.05584),
 (-15.86004, -48.04966),
 (-15.85732, -48.0433),
 (-15.84406, -48.05021),
 (-15.84014, -48.04322),
 (-15.84154, -48.04254),
 (-15.84214, -48.04362)]

In [6]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [7]:
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = -48.059358,-16.000091,-48.043619,-15.842141
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

{'route': [(-16.00009, -48.05936),
  (-15.9975, -48.05801),
  (-15.99324, -48.05046),
  (-15.96603, -48.01943),
  (-15.9477, -48.03621),
  (-15.9389, -48.04194),
  (-15.88007, -48.06122),
  (-15.86803, -48.05558),
  (-15.86528, -48.05584),
  (-15.86004, -48.04966),
  (-15.85732, -48.0433),
  (-15.84406, -48.05021),
  (-15.84014, -48.04322),
  (-15.84154, -48.04254),
  (-15.84214, -48.04362)],
 'start_point': [-16.000091, -48.05936],
 'end_point': [-15.842136, -48.043621],
 'distance': 22854.3}

In [8]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [9]:
get_map(test_route)

In [17]:
lon1 = np.random.uniform(-48.0,-49.0, 100).round(6)
lon2 = np.random.uniform(-48,-49.0, 100).round(6)
lat1 = np.random.uniform(-15.8,-16.0, 100).round(6)
lat2 = np.random.uniform(-15.8,-16.0, 100).round(6)
df = pd.DataFrame({'pickup_lon': lon1,
              'pickup_lat': lat1,
              'dropoff_lon': lon2,
              'dropoff_lat': lat2,
             })

In [18]:
df.head()

,pickup_lon,pickup_lat,dropoff_lon,dropoff_lat
0,-48.132756,-15.894332,-48.637013,-15.987327
1,-48.446214,-15.905397,-48.598804,-15.969770
2,-48.253975,-15.899657,-48.448997,-15.976297
3,-48.238285,-15.880506,-48.162568,-15.955455
4,-48.149664,-15.929175,-48.285919,-15.838962


In [19]:
%%time
df['routes'] = df.apply(lambda x: get_route(x['pickup_lon'], 
                                            x['pickup_lat'],
                                            x['dropoff_lon'], 
                                            x['dropoff_lat']), axis=1)

CPU times: user 123 ms, sys: 16.3 ms, total: 139 ms
Wall time: 239 ms


In [13]:
get_map(df.loc[45,'routes'])